In [56]:
import os
from datetime import timedelta
from snowflake.snowpark.exceptions import SnowparkSQLException
from snowflake.connector import connect
from dataframe_p.sf_session import session
from snowflake.snowpark import functions as F

from snowflake.core import Root
from snowflake.core.task import StoredProcedureCall, Task
from snowflake.core.task.dagv1 import DAGOperation, DAG, DAGTask

from dotenv import load_dotenv
load_dotenv('.env')


False

In [57]:

CONNECTION_PARAMETERS = {
    "account": "LNZNUFZ-YK66707",
    "user": "ABHINAVKS",
    "password": "Abhinav@200700",
    "role": "ACCOUNTADMIN",
    "database": "tasty_bytes_sample_data",
    "schema": "raw_pos"
}


# Create connection
# Step 2: Connect to Snowflake
connection = connect(**CONNECTION_PARAMETERS)

# Step 3: Create Root object (this expects a Connection, not a string!)
api_root = Root(connection)

# Step 4: Access your schema
schema = api_root.databases["tasty_bytes_sample_data"].schemas["raw_pos"]
print("Success!!")

Success!!


In [58]:
def task1(session : Session) -> str:
    print('task1')
    return 'task1 is runnig!'
def task2(session : Session) -> str:
    print('task2')
    try:
        tdf = session.sql("SELECT 1/0;")
        tdf.collect()
        
    except Exception as e:
        df = session.sql("""CALL SYSTEM$SEND_EMAIL(
        'send_email_int',
        'abhinav.singh43@infosys.com',
        'Email Alert: Task_c2 has failed.',
        'Task_c2 has  failed.');""")
        df.collect()
        raise SnowparkSQLException("Forcing failure of task2 due to insert error.")
    
def task3(session : Session) -> str:
    print('task3')
    return 'task3 is running'
def task4(session : Session) -> str:
    print('task4')
    return 'task4 is running'

In [59]:
dag_name = "test_dag"
dag = DAG(dag_name, schedule=timedelta(minutes=1))
with dag:
    dag_task1 = DAGTask("task1", StoredProcedureCall(task1, stage_location="@test_dag", packages=["snowflake-snowpark-python"]))
    dag_task2 = DAGTask("task2", StoredProcedureCall(task2, stage_location="@test_dag", packages=["snowflake-snowpark-python"]))
    dag_task3 = DAGTask("task3", StoredProcedureCall(task3, stage_location="@test_dag", packages=["snowflake-snowpark-python"]))
    dag_task4 = DAGTask("task4", StoredProcedureCall(task4, stage_location="@test_dag", packages=["snowflake-snowpark-python"]))
    dag_task2 >> dag_task1  # task1 → task2
    dag_task3 >> dag_task1  # task1 → task3
    dag_task4 >> dag_task2  # task2 → task4
    dag_task4 >> dag_task3
dag_op = DAGOperation(schema)
dag_op.deploy(dag, mode="orReplace")


The version of package 'snowflake-snowpark-python' in the local environment is 1.30.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-snowpark-python' in the local environment is 1.30.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-snowpark-python' in the local environment is 1.30.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-snowpark-python' in the local environment is 1.30.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Y

In [60]:
cur = connection.cursor()
cur.execute("select * from TASTY_BYTES_SAMPLE_DATA.RAW_POS.CAR_SALES;")
results = cur.fetchall()

for row in results:
    print(row)

('{\n  "customer": [\n    {\n      "address": "San Francisco, CA",\n      "name": "Joyce Ridgely",\n      "phone": "16504378889"\n    }\n  ],\n  "date": "2017-04-28",\n  "dealership": "Valley View Auto Sales",\n  "salesperson": {\n    "id": "55",\n    "name": "Frank Beasley"\n  },\n  "vehicle": [\n    {\n      "extras": [\n        "ext warranty",\n        "paint protection"\n      ],\n      "make": "Honda",\n      "model": "Civic",\n      "price": "20275",\n      "year": "2017"\n    }\n  ]\n}',)
('{\n  "customer": [\n    {\n      "address": "New York, NY",\n      "name": "Bradley Greenbloom",\n      "phone": "12127593751"\n    }\n  ],\n  "date": "2017-04-28",\n  "dealership": "Tindel Toyota",\n  "salesperson": {\n    "id": "274",\n    "name": "Greg Northrup"\n  },\n  "vehicle": [\n    {\n      "extras": [\n        "ext warranty",\n        "rust proofing",\n        "fabric protection"\n      ],\n      "make": "Toyota",\n      "model": "Camry",\n      "price": "23500",\n      "year": "20